## Generating constraint and violation data for building services on top of the project

In [2]:
from pathlib import Path
import os
from semconstmining.parsing.label_parser.nlp_helper import NlpHelper
from semconstmining.config import Config
from semconstmining.main import get_resource_handler, get_or_mine_constraints
from semconstmining.declare.enums import Template

MODEL_COLLECTION = "semantic_sap_sam_filtered"
config = Config(Path(os.getcwd()).parents[1].resolve(), MODEL_COLLECTION)
config.CONSTRAINT_TYPES_TO_IGNORE.remove(Template.NOT_CO_EXISTENCE.templ_str)

/Users/adrianrebmann/Develop/Best-Practice Conf./content/dev/semantic-constraint-miner/env/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [19]:
nlp_helper = NlpHelper(config)
resource_handler = get_resource_handler(config, nlp_helper)
all_constraints = get_or_mine_constraints(config, resource_handler, min_support=1)

[2024-07-12 15:14:40,012] p19741 {keyedvectors.py:2047} INFO - loading projection weights from /Users/adrianrebmann/gensim-data/glove-wiki-gigaword-50/glove-wiki-gigaword-50.gz
[2024-07-12 15:14:53,111] p19741 {utils.py:448} INFO - KeyedVectors lifecycle event {'msg': 'loaded (400000, 50) matrix of type float32 from /Users/adrianrebmann/gensim-data/glove-wiki-gigaword-50/glove-wiki-gigaword-50.gz', 'binary': False, 'encoding': 'utf8', 'datetime': '2024-07-12T15:14:53.111936', 'gensim': '4.3.2', 'python': '3.10.13 (main, Aug 24 2023, 12:59:26) [Clang 15.0.0 (clang-1500.1.0.2.5)]', 'platform': 'macOS-14.5-x86_64-i386-64bit', 'event': 'load_word2vec_format'}
[2024-07-12 15:14:53,113] p19741 {nlp_helper.py:95} INFO - Loaded 0 known embeddings and 0 known similarities
[2024-07-12 15:14:53,116] p19741 {resource_handler.py:113} INFO - Loading elements from /Users/adrianrebmann/Develop/Best-Practice Conf./content/dev/data/interim/semantic_sap_sam_filtered_bpmn_elements.pkl.
[2024-07-12 15:14:5

In [22]:
all_constraints#[all_constraints["Level"]=="Object"]

,obs_id,Level,Object,constraint_string,op_type,dictionary,data_object,template,left_op,right_op,model_id,model_name,support,redundant,nat_lang_template,ltl
7,8ac467ac-7f59-427b-aed2-6c80ac47a1c8,Activity,,"Alternate Succession[record order, prepare ord...",Binary,{},{},Alternate Succession,record order,prepare order for shiping,1bb03792264e4bd6914da6aafe094f34,BPMN,1,False,{1} and {2} occur if and only if the latter fo...,8ac467ac-7f59-427b-aed2-6c80ac47a1c8 := (G((re...
9,fde18428-9762-48b0-8112-09978bb2abdd,Activity,,"Succession[record order, prepare order for shi...",Binary,{},{},Succession,record order,prepare order for shiping,1bb03792264e4bd6914da6aafe094f34,BPMN,1,False,{1} occurs if and only if it is followed by {2},fde18428-9762-48b0-8112-09978bb2abdd := (G((re...
14,48c81b73-8ec7-403f-b652-c94fefa04309,Activity,,Alternate Precedence[check fulfillment request...,Binary,{},{},Alternate Precedence,check fulfillment request,place prescription in immediate box,1bd1ef6b99d6492c9d533f9120462f18,Jane Doe - Receive prescription,1,False,"each time {2} occurs, it is preceded by {1} an...",48c81b73-8ec7-403f-b652-c94fefa04309 := (((~(p...
15,58bfa395-7590-4014-9bb6-07503690a0e2,Activity,,"Precedence[check fulfillment request, place pr...",Binary,{},{},Precedence,check fulfillment request,place prescription in labeled box,1bd1ef6b99d6492c9d533f9120462f18,Jane Doe - Receive prescription,1,False,{2} occurs only if it is preceded by {1},58bfa395-7590-4014-9bb6-07503690a0e2 := ((~(pl...
17,2e176e59-fec6-4317-9493-16ad78de6f5a,Activity,,Exclusive Choice[place prescription in immedia...,Binary,{},{},Exclusive Choice,place prescription in immediate box,place prescription in labeled box,1bd1ef6b99d6492c9d533f9120462f18,Jane Doe - Receive prescription,1,False,"{1} or {2} occurs, but never both in the same ...",2e176e59-fec6-4317-9493-16ad78de6f5a := ((F(pl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109065,7fff701f-f442-43f6-a6db-3a23b50eca5b,Activity,,Succession[enter field of interest money exper...,Binary,{},{},Succession,enter field of interest money experience and h...,receive offer,e8f3d14581d74d578b88f55cf9988ff3,task2_E_j04_a12345678,1,False,{1} occurs if and only if it is followed by {2},7fff701f-f442-43f6-a6db-3a23b50eca5b := (G((en...
109067,13739846-14cd-4b89-b217-def8b3a1fe8e,Activity,,"Succession[receive tweets, show to friendlist]...",Binary,{},{},Succession,receive tweets,show to friendlist,e8f3d14581d74d578b88f55cf9988ff3,task2_E_j04_a12345678,1,False,{1} occurs if and only if it is followed by {2},13739846-14cd-4b89-b217-def8b3a1fe8e := (G((re...
109068,667d75fb-f47c-477d-9d33-efeab8c8c68a,Activity,,Alternate Succession[enter field of interest m...,Binary,{},{},Alternate Succession,enter field of interest money experience and h...,write update min week,e8f3d14581d74d578b88f55cf9988ff3,task2_E_j04_a12345678,1,False,{1} and {2} occur if and only if the latter fo...,667d75fb-f47c-477d-9d33-efeab8c8c68a := (G((en...
109069,b5efb917-81ac-4e62-bb3d-3c826397ec83,Activity,,Succession[enter field of interest money exper...,Binary,{},{},Succession,enter field of interest money experience and h...,write update min week,e8f3d14581d74d578b88f55cf9988ff3,task2_E_j04_a12345678,1,False,{1} occurs if and only if it is followed by {2},b5efb917-81ac-4e62-bb3d-3c826397ec83 := (G((en...


In [15]:
from semconstmining.main import get_or_mine_constraints, get_log_and_info, recommend_constraints_for_log, get_parts_of_constraints, compute_relevance_for_log
from semconstmining.selection.instantiation.filter_config import FilterConfig
from semconstmining.selection.instantiation.recommendation_config import RecommendationConfig

filt_config = FilterConfig(config)
nlp_helper.pre_compute_embeddings(sentences=get_parts_of_constraints(config, all_constraints))
rec_config = RecommendationConfig(config, semantic_weight=0.9, top_k=250)
event_log, log_info = get_log_and_info(config, nlp_helper, "BPI_Challenge_2019-3-w-after.xes")
recommended_constraints = compute_relevance_for_log(config, all_constraints, nlp_helper, "BPI_Challenge_2019-3-w-after.xes", pd_log=event_log, precompute=True)
recommended_constraints = recommend_constraints_for_log(config, rec_config, recommended_constraints, nlp_helper, "BPI_Challenge_2019-3-w-after.xes", pd_log=event_log)
recommended_constraints

Batches:   0%|          | 0/356 [00:00<?, ?it/s]

parsing log, completed traces ::   0%|          | 0/4 [00:00<?, ?it/s]

[2024-07-12 15:02:26,170] p19741 {relevance_computer.py:21} INFO - Computing relevance for 58373 constraints
[2024-07-12 15:02:26,182] p19741 {relevance_computer.py:25} INFO - Loading relevance from semantic_sap_sam_filtered_relevance_None.pkl
[2024-07-12 15:02:27,141] p19741 {main.py:288} INFO - Relevance computation took 0.9710779190063477 seconds
[2024-07-12 15:02:31,262] p19741 {constraintfitter.py:33} INFO - Number of fitted constraints: 11607
[2024-07-12 15:02:32,003] p19741 {constraintrecommender.py:124} INFO - Computed relevance scores.
[2024-07-12 15:02:32,009] p19741 {constraintrecommender.py:126} INFO - Recommended 9693 constraints


,index,Level,Object,constraint_string,op_type,dictionary,data_object,template,left_op,right_op,...,model_name,support,redundant,nat_lang_template,ltl,individual_relevance_scores,semantic_based_relevance,fitted_obs_id,relevance_score,activation
obs_id,,,,,,,,,,,,,,,,,,,,,


In [29]:
import pandas as pd
declare_consts = pd.read_pickle(config.DATA_INTERIM / "semantic_sap_sam_filtered_bpmn_declare_const.pkl")
declare_consts[declare_consts["Level"]=="Object"]
declare_consts["Level"].unique()

array(['Activity', 'Resource'], dtype=object)

In [3]:
import pandas as pd
fitted = pd.read_pickle(config.DATA_INTERIM / "BPI_Challenge_2019-3-w-after.xes-constraints_with_relevance.pkl")
fitted

,obs_id,Level,Object,constraint_string,op_type,dictionary,data_object,template,left_op,right_op,model_id,model_name,support,redundant,nat_lang_template,ltl,individual_relevance_scores,semantic_based_relevance
1,5e00ee06-e8f8-411b-899f-4fa3bf2ef589,Activity,,"Alternate Succession[record order, prepare ord...",Binary,{},{},Alternate Succession,record order,prepare order for shiping,1bb03792264e4bd6914da6aafe094f34,BPMN,1,False,{1} and {2} occur if and only if the latter fo...,5e00ee06-e8f8-411b-899f-4fa3bf2ef589 := (G((re...,{'left_op': {'srm created': 0.0679610371589660...,0.582095
10,be064bea-bb01-4de5-a3be-3ec7b2af0b97,Activity,,"Succession[record order, prepare order for shi...",Binary,{},{},Succession,record order,prepare order for shiping,1bb03792264e4bd6914da6aafe094f34,BPMN,1,False,{1} occurs if and only if it is followed by {2},be064bea-bb01-4de5-a3be-3ec7b2af0b97 := (G((re...,{'left_op': {'srm created': 0.0679610371589660...,0.582095
15,d6155b2f-3a5c-4dff-8b99-214e661b8fb8,Activity,,"Precedence[check fulfillment request, place pr...",Binary,{},{},Precedence,check fulfillment request,place prescription in labeled box,1bd1ef6b99d6492c9d533f9120462f18,Jane Doe - Receive prescription,1,False,{2} occurs only if it is preceded by {1},d6155b2f-3a5c-4dff-8b99-214e661b8fb8 := ((~(pl...,{'left_op': {'srm created': 0.2231534123420715...,0.470478
16,4825b488-3b09-4408-bfa9-0860d6ed379e,Activity,,Alternate Precedence[check fulfillment request...,Binary,{},{},Alternate Precedence,check fulfillment request,place prescription in labeled box,1bd1ef6b99d6492c9d533f9120462f18,Jane Doe - Receive prescription,1,False,"each time {2} occurs, it is preceded by {1} an...",4825b488-3b09-4408-bfa9-0860d6ed379e := (((~(p...,{'left_op': {'srm created': 0.2231534123420715...,0.470478
17,4ba477f5-2605-4bec-9cc9-47cac3cb16d8,Activity,,"Precedence[check fulfillment request, place pr...",Binary,{},{},Precedence,check fulfillment request,place prescription in immediate box,1bd1ef6b99d6492c9d533f9120462f18,Jane Doe - Receive prescription,1,False,{2} occurs only if it is preceded by {1},4ba477f5-2605-4bec-9cc9-47cac3cb16d8 := ((~(pl...,{'left_op': {'srm created': 0.2231534123420715...,0.470478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166200,8f9bf953-1670-4545-9c52-127f94a14538,Multi-object,,"Succession[field of interest money experience,...",Binary,{},{},Succession,field of interest money experience,update min week,e8f3d14581d74d578b88f55cf9988ff3,task2_E_j04_a12345678,1,False,{1} occurs if and only if it is followed by {2},8f9bf953-1670-4545-9c52-127f94a14538 := (G((fi...,{'left_op': {'srm transfer': 0.062952831387519...,0.285482
166201,b2c138b4-7928-4e26-95d4-9ed53d481e05,Multi-object,,"Succession[field of interest money experience,...",Binary,{},{},Succession,field of interest money experience,recommendation,e8f3d14581d74d578b88f55cf9988ff3,task2_E_j04_a12345678,1,False,{1} occurs if and only if it is followed by {2},b2c138b4-7928-4e26-95d4-9ed53d481e05 := (G((fi...,{'left_op': {'srm transfer': 0.062952831387519...,0.292572
166202,b14a16ea-f761-4bbe-a168-3cab138cb366,Multi-object,,"Succession[update min week, recommendation] | | |",Binary,{},{},Succession,update min week,recommendation,e8f3d14581d74d578b88f55cf9988ff3,task2_E_j04_a12345678,1,False,{1} occurs if and only if it is followed by {2},b14a16ea-f761-4bbe-a168-3cab138cb366 := (G((up...,{'left_op': {'srm transfer': 0.110300213098526...,0.292572
166203,0463659f-50e0-4ae9-833e-205c2e933537,Multi-object,,Alternate Succession[field of interest money e...,Binary,{},{},Alternate Succession,field of interest money experience,update min week,e8f3d14581d74d578b88f55cf9988ff3,task2_E_j04_a12345678,1,False,{1} and {2} occur if and only if the latter fo...,0463659f-50e0-4ae9-833e-205c2e933537 := (G((fi...,{'left_op': {'srm transfer': 0.062952831387519...,0.285482


In [ ]:
# Export
all_constraints.to_csv("example_constraints.csv")

### Creation violation data

In [ ]:
from semconstmining.log.loghandler import LogHandler
from semconstmining.log.loginfo import LogInfo
from semconstmining.log.logstats import LogStats
from semconstmining.selection.instantiation.filter_config import FilterConfig
from semconstmining.selection.instantiation.recommendation_config import RecommendationConfig
from semconstmining.main import get_parts_of_constraints, compute_relevance_for_log, get_log_and_info, recommend_constraints_for_log, check_constraints, get_violation_to_cases
from semconstmining.selection.instantiation.constraintfilter import ConstraintFilter
from semconstmining.selection.instantiation.constraintrecommender import ConstraintRecommender
from os.path import exists
import pandas as pd


CURRENT_LOG_FILE = "BPI_Challenge_2019-3-w-after.xes"

filter_config = FilterConfig(config)
recommender_config = RecommendationConfig(config, top_k=250)

nlp_helper.pre_compute_embeddings(sentences=get_parts_of_constraints(config, all_constraints))

const_filter = ConstraintFilter(config, filter_config, resource_handler)
filtered_constraints = const_filter.filter_constraints(all_constraints)
event_log, log_info = get_log_and_info(config, nlp_helper, CURRENT_LOG_FILE)

# Log-specific constraint recommendation
if not exists(config.DATA_INTERIM / (CURRENT_LOG_FILE + "-constraints_with_relevance.pkl")):
    filtered_constraints = compute_relevance_for_log(config, filtered_constraints, nlp_helper, CURRENT_LOG_FILE,
                                                     pd_log=event_log, precompute=True)
    filtered_constraints.to_pickle(config.DATA_INTERIM / (CURRENT_LOG_FILE + "-constraints_with_relevance.pkl"))
else:
    filtered_constraints = pd.read_pickle(config.DATA_INTERIM / (CURRENT_LOG_FILE + "-constraints_with_relevance.pkl"))
recommended_constraints = recommend_constraints_for_log(config, recommender_config, filtered_constraints,
                                                        nlp_helper,
                                                        CURRENT_LOG_FILE, pd_log=event_log)
consistent_recommended_constraints = recommended_constraints
violations = check_constraints(config, CURRENT_LOG_FILE, consistent_recommended_constraints, nlp_helper, pd_log=event_log)
violations_to_cases = get_violation_to_cases(config, violations)
violation_df = pd.DataFrame.from_records(
    [{"violation": violation, "num_violations": len(cases), "cases": cases} for violation, cases in
     violations_to_cases.items()])

In [ ]:
violation_df

In [ ]:
# Export
violation_df.to_pickle("example_violations.pkl")